 ##Logistic Regression


In [1]:
from pathlib import Path
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np
from sklearn.preprocessing import StandardScaler
pd.options.display.float_format = '{:.0f}'.format

In [2]:
# data = Path('../Resources/diabetes.csv')

start_data = pd.read_csv('Resources/cleaned.csv')
start_data.head()
start_data.columns


boxOffice = start_data['BoxOffice_Inf_Adj'].dropna()
sub25 = boxOffice[boxOffice >= 55930]
# sub50 = boxOffice[boxOffice[boxOffice > 55930 & boxOffice < 29832937]
# box75 = boxOffice[boxOffice <= 29832937]
# sub25.count() = 3453


In [3]:
#Creating a BoxOffice Category
q1, q2, q3 = start_data['BoxOffice_Inf_Adj'].quantile([0.25, 0.50, 0.75])


conditions = [
    start_data['BoxOffice_Inf_Adj'].isna(),                       
    start_data['BoxOffice_Inf_Adj'] < q1,                           
    (start_data['BoxOffice_Inf_Adj'] >= q1) & (start_data['BoxOffice_Inf_Adj'] < q2),    
    (start_data['BoxOffice_Inf_Adj'] >= q2) & (start_data['BoxOffice_Inf_Adj'] < q3),   
    start_data['BoxOffice_Inf_Adj'] >= q3        
]

choices = [
    'Unrated',
    'Quart_1',
    'Quart_2',
    'Quart_3',
    'Quart_4'
]

start_data['BoxOfficeCategory'] = np.select(conditions, choices)
start_data.groupby('BoxOfficeCategory').count()

,Title,Rated,Released,Runtime,Genre,Director,Actors,Language,Country,Awards,...,SPORT,WAR,MUSICAL,WESTERN,NEWS,REALITY-TV,Unnamed: 44,GAME-SHOW,TALK-SHOW,BoxOffice_Inf_Adj
BoxOfficeCategory,,,,,,,,,,,,,,,,,,,,,
Quart_1,1151,1151,1151,1151,1151,1151,1123,1143,1151,723,...,28,10,8,14,10,0,0,0,0,1151
Quart_2,1151,1151,1151,1151,1151,1151,1137,1145,1151,888,...,36,15,10,15,13,0,0,0,0,1151
Quart_3,1151,1151,1151,1151,1151,1151,1149,1146,1151,996,...,38,17,7,9,1,0,0,0,0,1151
Quart_4,1152,1152,1152,1152,1152,1152,1152,1152,1152,1127,...,21,8,10,4,0,0,0,0,0,1152
Unrated,13373,13373,13373,13373,13365,13292,12912,13132,13373,5320,...,334,123,121,126,143,8,8,1,1,0


In [4]:
# start_data[[ 'imdbRating','Good_Movie']][start_data['Good_Movie'] == True]
# start_data['Runtime'] = start_data['Runtime'].str.replace(' min', '', regex=False).astype(int)
start_data['imdbVotes'] = start_data['imdbVotes'].str.replace(',', '', regex=False).astype(float)




In [5]:
model_data = start_data[start_data['R_Year'] >= 2010]

model_data = model_data[['Director','imdbVotes','BoxOfficeCategory','Rated','R_Year','R_Month','R_Day','Runtime','Good_Movie',
       'DRAMA', 'HORROR', 'MUSIC', 'THRILLER',
       'DOCUMENTARY', 'ANIMATION', 'ACTION', 'ADVENTURE', 'BIOGRAPHY',
       'COMEDY', 'FANTASY', 'MYSTERY', 'ROMANCE', 'SCI-FI', 'CRIME', 'FAMILY',
       'HISTORY', 'SPORT', 'WAR', 'MUSICAL', 'WESTERN', 'NEWS']].copy()


model_data = model_data.fillna(0)
model_data

,Director,imdbVotes,BoxOfficeCategory,Rated,R_Year,R_Month,R_Day,Runtime,Good_Movie,DRAMA,...,ROMANCE,SCI-FI,CRIME,FAMILY,HISTORY,SPORT,WAR,MUSICAL,WESTERN,NEWS
0,Orson Welles,8188,Unrated,R,2018,11,2,122,False,1,...,0,0,0,0,0,0,0,0,0,0
1,André Szöts,1801,Unrated,Not Rated,2021,1,8,74,False,0,...,0,0,0,0,0,0,0,0,0,0
2,John Bailey,72,Unrated,PG-13,2019,11,13,101,True,1,...,0,0,0,0,0,0,0,0,0,0
3,John Mulholland,100,Unrated,Unrated,2013,9,27,120,False,0,...,0,0,0,0,0,0,0,0,0,0
4,Stuart Paul,151,Unrated,TV-MA,2018,11,6,104,False,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17973,Joe Begos,9831,Unrated,Not Rated,2020,2,14,92,False,0,...,0,0,1,0,0,0,0,0,0,0
17974,Geeta Malik,893,Quart_2,PG-13,2022,2,3,101,False,0,...,0,0,0,0,0,0,0,0,0,0
17975,Robert Fernandez,921,Quart_3,PG,2019,4,18,108,False,0,...,0,0,0,1,0,0,0,0,0,0
17976,Michael Dowse,14780,Unrated,TV-MA,2020,4,3,88,False,0,...,0,0,1,0,0,0,0,0,0,0


In [6]:
model_data = pd.get_dummies(model_data, columns=['Rated','R_Year','BoxOfficeCategory','R_Month','R_Day','Director'])
model_data


,imdbVotes,Runtime,Good_Movie,DRAMA,HORROR,MUSIC,THRILLER,DOCUMENTARY,ANIMATION,ACTION,...,Director_Zoe Quist,Director_Zoran Lisinac,Director_Zoë Kravitz,Director_Zu Quirke,Director_Álex de la Iglesia,Director_Álvaro Cáceres,Director_Èric Boadella,Director_Étienne Faure,Director_Étienne Sauret,Director_Óscar Ruiz Navia
0,8188,122,False,1,0,0,0,0,0,0,...,False,False,False,False,False,False,False,False,False,False
1,1801,74,False,0,1,1,1,0,0,0,...,False,False,False,False,False,False,False,False,False,False
2,72,101,True,1,0,0,0,0,0,0,...,False,False,False,False,False,False,False,False,False,False
3,100,120,False,0,0,0,0,1,0,0,...,False,False,False,False,False,False,False,False,False,False
4,151,104,False,1,0,0,0,0,0,0,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17973,9831,92,False,0,1,0,0,0,0,1,...,False,False,False,False,False,False,False,False,False,False
17974,893,101,False,0,0,0,0,0,0,0,...,False,False,False,False,False,False,False,False,False,False
17975,921,108,False,0,0,0,0,0,1,0,...,False,False,False,False,False,False,False,False,False,False
17976,14780,88,False,0,0,0,0,0,0,1,...,False,False,False,False,False,False,False,False,False,False


In [15]:
scaler = StandardScaler()
y = model_data["Good_Movie"]
model_data[['imdbVotes','Runtime']] = scaler.fit_transform(model_data[['imdbVotes','Runtime']])
df = model_data.copy()


 ## Separate the Features (X) from the Target (y)

In [16]:

X = df.drop(columns="Good_Movie")

 ## Split our data into training and testing

In [17]:


from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1, 
                                                    stratify=y)
X_train.shape

(13482, 12457)

 ## Create a Logistic Regression Model

In [25]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(solver='lbfgs',
                                max_iter=2000,
                                random_state=1)
classifier.fit(X_train, y_train)

LogisticRegression(max_iter=2000, random_state=1)

 ## Fit (train) or model using the training data

 ## Score the model using the test data

In [26]:
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

Training Data Score: 0.9192998071502745
Testing Data Score: 0.8762794837561193


 ## Make predictions

In [27]:
predictions = classifier.predict(X_test)
results = pd.DataFrame({"Prediction": predictions, "Actual": y_test}).reset_index(drop=True)
results.head(10)

,Prediction,Actual
0,False,False
1,False,False
2,False,False
3,False,False
4,False,False
5,False,False
6,False,False
7,False,False
8,False,False
9,False,False


## Calculate the Accuracy Score

In [28]:
print(results.count())
print(results[results['Actual'] == True])

Prediction    4494
Actual        4494
dtype: int64
      Prediction  Actual
10         False    True
16         False    True
23         False    True
28         False    True
36          True    True
...          ...     ...
4425        True    True
4432        True    True
4465        True    True
4474       False    True
4480       False    True

[600 rows x 2 columns]


In [29]:
from sklearn.metrics import accuracy_score
# Display the accuracy score for the test dataset.
accuracy_score(y_test, predictions)

0.8762794837561193

In [30]:
confusion_matrix(y_test, predictions)

array([[3802,   92],
       [ 464,  136]])

In [31]:
target_names = ["Training", "Test"]
print(classification_report(y_test, predictions, target_names=target_names))

              precision    recall  f1-score   support

    Training       0.89      0.98      0.93      3894
        Test       0.60      0.23      0.33       600

    accuracy                           0.88      4494
   macro avg       0.74      0.60      0.63      4494
weighted avg       0.85      0.88      0.85      4494

